<img src="https://github.com/vhertel/drought-monitoring/blob/main/resources/header.png?raw=1" width="1000"/>

# Drought Monitoring

<img src="https://github.com/vhertel/drought-monitoring/blob/main/resources/example.png?raw=1" width="1000"/>

***

Spectral vegetation indices are among the most commonly used satellite data products for evaluation, monitoring, and measurement of vegetation cover, condition, biophysical processes, and changes. This [Recommended Practice](https://un-spider.org/advisory-support/recommended-practices) shows how to apply a multi-temporal analysis of the MODIS-based Standardized Vegetation Index (SVI) or Vegetation Condition Index (VCI) to support drought monitoring and early warning.

This Jupyter Notebook covers the full processing chain from data query and download up to the export of the final data products by utilizing open access MODIS data. The tool's workflow follows the UN-SPIDER Recommended Practice on [Drought Monitoring](https://www.un-spider.org/advisory-support/recommended-practices/recommended-practice-drought-monitoring-using-standard) and is illustrated in the chart below. After entering user specifications, MODIS data can directly be downloaded from [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/) (Application for Extracting and Exploring Analysis Ready Samples). Subsequently, the data is processed and stored in a variety of output formats.

<img src="https://github.com/vhertel/drought-monitoring/blob/main/resources/charts/chart0.png?raw=1" width="1000"/>

***

***File Structure***  
The Jupyter Notebook file constitutes the directory of origin. Additional data is contained in subfolders. MODIS data needs to be stored in subfolders called *'input/evi_data'* and *'input/pixel_reliability'*. If no image is provided, the subfolders will automatically be created when accessing and downloading data from [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/) through this tool. The processed data is stored in a subfolder called *'output'*.  

***Limitations***  
MODIS sensors are mounted on two satellite platforms: Terra (launched in December 1999) and Aqua (launched in May 2002). This means, the time series go back to 2000/2002, which is not very long from a climatologic viewpoint. Since the SVI is using the mean and standard deviation of the time series, it is desirable to use a representative time span. It is, however, possible to extend the time series with AVHRR data provided that a thorough inter-calibration between the two sensor systems is given.  
The SVI can only provide a relative comparison of the vegetation condition while the assessed deviation from the mean vegetation condition cannot be translated into an absolute deviation of for example the plant height. Neither can the SVI be interpreted for absolute quantification of agricultural damage.

***

## Initialization & User Input

<img src="https://github.com/vhertel/drought-monitoring/blob/main/resources/charts/chart1.png?raw=1" width="1000"/>

Please run the cell below for initialization and specify the working directory as well as whether data shall be downloaded from [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/).

In [ ]:
# Click to run

#####################################################
###################### IMPORTS ######################
#####################################################

import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd
import fnmatch
import imageio
import os
import re
import gc
import builtins
import warnings
import numpy as np
import ipyleaflet
import geopandas
import json
import ipywidgets
import functools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import datetime
from datetime import date
from matplotlib import colors
from ipyfilechooser import FileChooser
from osgeo import gdal
gdal.UseExceptions()
warnings.filterwarnings('ignore')

# If tqdm is installed it will be used to display a nice progress bar
try:
    from tqdm.autonotebook import tqdm
    import sys
    # Overload built-in enumerate function
    def enumerate(iterator, desc=None):
        return builtins.enumerate(tqdm(iterator, desc=desc, file=sys.stdout,
                                       leave=False))
except ImportError:
    tqdm = None
    # Overload built-in enumerate function
    def enumerate(iterator, desc=None):
        if desc is not None:
            print(desc)
        return builtins.enumerate(iterator)




####################################################
############### FUNCTION DEFINITIONS ###############
####################################################

# Searches the input string for a DOY
def get_doy(re_doy, string):
    """
    Searches the input string for a DOY, and if one is found it returns a tuple
    containing the DOY; else it returns None.

    :param re_doy: compiled re pattern used to match a DOY
    :param string: input string that will be searched for a DOY
    :return : if a DOY is found a tuple of the form
              (<int> year, <str> day of year), else None
    """
    search_doy = re_doy.search(string)
    if search_doy is None:  # Case where no doy was found
        return None
    doy = search_doy.group(1)
    year = int(doy[:4])  # Treat year values as integers
    day = doy[4:]  # Day values need to remain strings to be zero padded
    return year, day

# Checks that for each evi file there is a corresponding pixel reliability file
def check_prepare_files(evi_files, pr_files):
    """
    Checks that for each evi file there is a corresponding pixel reliability
    file. Then a dictionary is returned containing the sorted files lists for
    each DOY.

    :param evi_files: list of evi files to be processed, containing a tuples of
                       the form (<str> file_path, <int> year, <str> day)
    :param pr_files: list of pixel reliability files to be processed,
                     containing a tuples ofthe form
                     (<str> file_path, <int> year, <str> day)
    :return: dictionary where keys are the available DOYs and values are
             dictionaries for evi_files and pr_files countaining the sorted
             evi_files and sorted pr_files.
    """
    # Dictionary keys have properties similar enough to a mathematical set for
    # our purposes
    doy_dict = dict()

    for path, year, day in evi_files:
        try:
            doy_dict[day]['evi'].append((path, year, day))
        except KeyError:
            doy_dict[day] = dict()
            doy_dict[day]['evi'] = [(path, year, day)]

    for path, year, day in pr_files:
        try:
            doy_dict[day]['pr'].append((path, year, day))
        except KeyError:
            try:
                doy_dict[day]['pr'] = [(path, year, day)]
            except KeyError:
                doy_dict[day] = dict()
                doy_dict[day]['pr'] = [(path, year, day)]

    if len(evi_files) != len(pr_files):
        mismatch_doys = list()
        for day in doy_dict.keys():
            try:
                if len(doy_dict[day]['evi']) != len(doy_dict[day]['pr']):
                    mismatch_doys.append(day)
            except KeyError:
                mismatch_doys.append(day)
        mismatch_files = dict()
        for day in mismatch_doys:
            if any(i not in doy_dict[day]['pr'] for i in doy_dict[day]['evi']):
                evi_years = [i[1] for i in doy_dict[day]['evi']]
                pr_years = [i[1] for i in doy_dict[day]['pr']]
                mismatch_evi = [i for i in evi_years if i not in pr_years]
                mismatch_pr = [i for i in pr_years if i not in evi_years]

                # If the lists are empty future list comprehensions will fail
                # to correctly exclude all files
                if len(mismatch_evi) == 0:
                    mismatch_evi.append(-1)
                if len(mismatch_pr) == 0:
                    mismatch_pr.append(-1)

                mismatch_files[day] = dict()
                mismatch_files[day]['evi'] = [i[0] for i in doy_dict[day]['evi']
                                              if i[1] in mismatch_evi]

                mismatch_files[day]['pr'] = [i[0] for i in doy_dict[day]['pr']
                                             if i[1] in mismatch_pr]
        war_msg = "There is a mismatch in the number of evi data files and "
        war_msg += "pixel reliability files.\n"
        war_msg += "The the mismatched files for each doy are:\n"
        war_msg += str(mismatch_files)
        war_msg += "\n"
        war_msg += "These files will be ignored."
        warnings.warn(war_msg)

        for day, v in mismatch_files.items():
            for k_sub, v_sub in v.items():
                doy_dict[day][k_sub] = [i for i in doy_dict[day][k_sub]
                                        if i[0] not in v_sub]
    else:
        for day in doy_dict.keys():
            doy_dict[day]['evi'] = sorted(doy_dict[day]['evi'],
                                          key=lambda x: x[1])
            doy_dict[day]['pr'] = sorted(doy_dict[day]['pr'],
                                         key=lambda x: x[1])

    return doy_dict

# Loads all pixel reliability images into the cloud mask array
def load_cloud_mask(files_list, cloud_mask):
    """
    Loads all pixel reliability images into the cloud mask array.

    :param files_list: list of files to be processed, containing a tuples of
                       the form (<str> file_path, <int> year, <str> day)
    :param cloud_mask: matrix containing the processed pixel reliability data
    :return:
    """
    for i, image in enumerate(files_list, "Loading cloud_mask"):
        # Open the file from the pixel reliability file list
        dataset = gdal.Open(image[0])
        band = dataset.GetRasterBand(1)
        # del dataset
        data = band.ReadAsArray()

        # Write the data from each file to the array
        cloud_mask[:, :, i] = data

    return cloud_mask

# Filters the pixel reliability array
def prepare_cloud_mask(cloud_mask):
    """
    Filters the pixel reliability array.

    :param cloud_mask: pixel reliability array
    :return: filtered and rescaled pixel reliability matrix
    """
    # Exchange value 0 with value 1 (Pixels with value 0 and 1 are used)
    cloud_mask[cloud_mask == 0] = 1

    # Set values that are of no interest to us to NaN
    # cloud_mask[cloud_mask != 1] = np.nan
    cloud_mask[cloud_mask != 1] = np.nan

    # These are additional filters you may want to use
    # Set value 2 to NA (Snow and Ice)
    # cloud_mask[cloud_mask == 2] = np.nan

    # Set value 3 to NA (Cloud)
    # cloud_mask[cloud_mask == 3] = np.nan

    # Set no data value (= -1) to NA
    # cloud_mask[cloud_mask == -1] = np.nan

    # Set all values above 3 to NA
    # cloud_mask[cloud_mask > 3] = np.nan

    return cloud_mask

# Loads a single geotiff into the evi matrix
def load_evi(files_list, cloud_mask):
    """
    Loads a single geotiff into the evi matrix.

    :param files_list: list of files to be processed, containing a tuples of
                       the form (<str> file_path, <int> year, <str> day)
    :param cloud_mask: matrix containing the processed pixel reliability data
    :return: evi matrix with an additional geotiff of data
    """
    for i, image in enumerate(files_list, "Loading EVI"):
        # Open the file from the evi file list
        dataset = gdal.Open(image[0])
        band = dataset.GetRasterBand(1)
        data = band.ReadAsArray()

        # Apply the cloud mask and write the data from each file to the array
        # Note: the evi data is multiplied into the cloud_mask matrix in order
        # to save RAM and speed up the computation
        cloud_mask[:, :, i] *= data

    return cloud_mask

# Filters the evi to set all negative values to nan and rescales the data
def prepare_evi(evi):
    """
    Filters the evi to set all negative values to nan and rescales the data.

    :param evi: numpy array containing the results of multiplying the
                cloud_mask data with the evi data
    :return: filtered and rescaled input numpy array
    """
    # Set negative values to nan
    evi[evi < 0] = np.nan

    # Rescale the data
    evi *= 0.0001

    return evi

# Calculates the SVI
def calculate_svi(evi):
    """
    Calculates the SVI.

    The standard deviation is calculated using the following formula:
    std = sqrt((E[X ** 2] - E[X] ** 2) * (N / N - 1))
    std = sqrt((mean(x ** 2) - mean(x) ** 2) * (N / N - 1))

    Here N is the number of obervations, and the multiplier (N / N - 1) is the
    adjustment factor necessary to convert the standard deviation to an
    unbiased estimator of the variance of the infinite population.

    The SVI itself is calculated using the following formula:
    SVI = (EVI - mean(EVI)) / std(EVI)

    A lot of the operations are done in place, such as squaring the evi matrix,
    using it for something, and then taking the square root of it to get back
    the original values. This may seem inefficient however it is much faster
    than allocating large amount of memory and doing copy based operations.

    When it comes to numpy arrays, abbreviated code like this:
    a += b
    c /= d
    Means that the operations are performed in place.

    Whereas operations like this:
    a = a + b
    c = c / d
    First create a copy of the a and c arrays before performing the addition
    and division. This leads to additional memory and computational overhead
    due to creating a copy of the arrays.

    Since the input evi matrix can have only positive values squaring and
    taking the square root in place causes no computational errors.


    Note: The code for computing the mean and standard deviation is loosely
          based on the built-in numpy methods for nanmean and nanstd.

    :param evi: numpy array containing the geotiff processing results
    :return: evi array which has had its values replaced in-place with the SVI
             values
    """
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        # Allocate memory to where intermittent data will be saved
        shape = (evi.shape[0], evi.shape[1], 1)
        num_non_nan = np.zeros(shape, dtype=np.float64)
        sum_ = np.zeros(shape, dtype=np.float64)
        sum_squares = np.zeros(shape, dtype=np.float64)

        # Create a mask that will say where nan values are, then negate it so
        # it shows where the non nan values are
        mask = ~np.isnan(evi)

        # Sum the non nan values
        # Note: setting keepdims to True will allow to do efficient array wide
        # broadcasting in future calculations (this greatly increases
        # computation speed and reduces memory usage)
        np.sum(mask, axis=2, out=num_non_nan, dtype=np.intp, keepdims=True)

        # Negate the mask so it shows where the nan values are
        np.invert(mask, out=mask)

        # Substitute nan values with 0 in place
        evi[mask] = 0

        # Calculate the sum
        np.sum(evi, axis=2, out=sum_, dtype=np.float64, keepdims=True)

        # Square all values in place
        np.square(evi, out=evi)

        # Calculate the sum of squares
        np.sum(evi, axis=2, out=sum_squares, dtype=np.float64, keepdims=True)

        # Take the square root of evi to get back all the original values
        np.sqrt(evi, out=evi)

        # Compute the mean by dividing the sum_ values in place
        evi_mean = sum_  # Set easy to understand pointer
        evi_mean /= num_non_nan  # Divide in place

        # Square the mean in place
        np.square(evi_mean, out=evi_mean)

        # Compute the standard deviation in place (saving it in sum_squares)
        evi_std = sum_squares  # Set easy to understand pointer
        evi_std /= num_non_nan
        evi_std -= evi_mean

        # Adjust the standard deviation to be an unbiased estimator of the
        # variance of the infinite population
        evi_std *= num_non_nan
        num_non_nan -= 1
        num_non_nan[num_non_nan <= 0] = np.nan  # Avoid zero division errors

        # Finish calculating the standard deviation
        evi_std /= num_non_nan
        np.sqrt(evi_std, out=evi_std)

        # Take the square root in place to get back original mean
        np.sqrt(evi_mean, out=evi_mean)

        # Replace 0 with nan in the evi array
        evi[mask] = np.nan

        # Calculate the SVI
        # Note: the svi is saved into the evi matrix in order to save RAM
        svi = evi
        svi -= evi_mean
        svi /= evi_std

        return svi
    
# Calculates the VCI
def calculate_vci(evi):
    """
    Calculates the VCI.

    The VCI is calculated using the following formula:
    VCI = ((EVI - min(EVI)) / (max(EVI) - min(EVI))) * 100

    A lot of the operations are done in place, such as squaring the evi matrix,
    using it for something, and then taking the square root of it to get back
    the original values. This may seem inefficient however it is much faster
    than allocating large amount of memory and doing copy based operations.

    When it comes to numpy arrays, abbreviated code like this:
    a += b
    c /= d
    Means that the operations are performed in place.

    Whereas operations like this:
    a = a + b
    c = c / d
    First create a copy of the a and c arrays before performing the addition
    and division. This leads to additional memory and computational overhead
    due to creating a copy of the arrays.

    Since the input evi matrix can have only positive values squaring and
    taking the square root in place causes no computational errors.


    Note: The code for computing the mean and standard deviation is loosely
          based on the built-in numpy methods for nanmean and nanstd.

    :param evi: numpy array containing the geotiff processing results
    :return: evi array which has had its values replaced in-place with the VCI
             values
    """
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        # Allocate memory to where intermittent data will be saved
        shape = (evi.shape[0], evi.shape[1], 1)
        min_ = np.zeros(shape, dtype=np.float64)
        max_ = np.zeros(shape, dtype=np.float64)

        # Create a mask that will say where nan values are, then negate it so
        # it shows where the non nan values are
        # mask = np.isnan(evi)

        # Substitute nan values with 0 in place
        # evi[mask] = 0

        # Find the minimum values
        np.fmin.reduce(evi, axis=2, out=min_, keepdims=True)

        # Find the maximum values
        np.fmax.reduce(evi, axis=2, out=max_, keepdims=True)

        # Calculate the VCI
        # Note: the vci is saved into the evi matrix in order to save RAM
        vci = evi
        vci -= min_
        max_ -= min_
        vci /= max_
        vci *= 100

        return vci

# Generate an output png image representing the SVI for the input map area
def generate_SVI_png(file_name, data, breaks, extent):
    """
    Generate an output png image representing the SVI for the input map area.

    :param file_name: <str> file name to be used when saving the png image
    :param data: numpy view onto the svi for a single year
    :param breaks: standard deviation values that should be used as breaks in
                   the outputted png images
    :param extent: tuple containing data for how to modify the output graphic
                   in order for it to scale correctly
    :return:
    """
    bounds = breaks  # Set easy to understand pointer

    # Define the size of the figure (in inches)
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.title(file_name)
    cmap = colors.ListedColormap(['#4C0E0D', '#E72223', '#F19069', '#F9F6C6',
                                  '#64B14B', '#04984A', '#00320E'])
    norm = colors.BoundaryNorm(bounds, cmap.N)
    cax = ax.imshow(data, cmap=cmap, norm=norm, extent=extent)
    fig.colorbar(cax, cmap=cmap, norm=norm, boundaries=bounds, ticks=breaks)

    plt.savefig(os.path.join(path_png, file_name + ".png"), dpi=100)
    plt.close()

# Generate an output png image representing the VCI for the input map area
def generate_VCI_png(file_name, data, extent):
    """
    Generate an output png image representing the VCI for the input map area.

    :param file_name: <str> file name to be used when saving the png image
    :param data: numpy view onto the vci for a single year
    :param extent: tuple containing data for how to modify the output graphic
                   in order for it to scale correctly
    :return:
    """
    # Define the size of the figure (in inches)
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.title(file_name)
    cmap = colors.ListedColormap(['#8B0000', '#FF4500', '#FFFF00', '#9ACD32',
                                  '#008000'])
    cax = ax.imshow(data, cmap=cmap, extent=extent)
    fig.colorbar(cax, cmap=cmap)

    plt.savefig(os.path.join(path_png, file_name + ".png"), dpi=100)
    plt.close()

# Generate an output geotiff image representing the SVI/VCI for the input map area
def generate_geotiff(file_name, data, geo_transform, projection):
    """
    Generate an output geotiff image representing the SVI/VCI for the input map
    area.

    :param file_name: <str> file name to be used when saving the tif image
    :param data: numpy view onto the svi/vci for a single year
    :param geo_transform: geo transform data to be used in saving the tif image
    :param projection: projection data to be used in saving the tif image
    :return:
    """
    # Set geotiff output path
    geotiff_path = os.path.join(path_tif, file_name + ".tif")

    # Read columns from data array
    cols = data.shape[1]
    # Read rows from data array
    rows = data.shape[0]

    # Set the driver to Geotiff
    driver = gdal.GetDriverByName('GTiff')
    # Create raster with shape of array as float64
    out_raster = driver.Create(geotiff_path, cols, rows, 1, gdal.GDT_Float64)
    # Read geo information from input file
    out_raster.SetGeoTransform(geo_transform)
    # Read band
    out_band = out_raster.GetRasterBand(1)
    # Set no data value to numpy's nan
    out_band.SetNoDataValue(np.nan)
    out_band.WriteArray(data)
    # Set the projection according to the input file projection
    out_raster.SetProjection(projection)
    out_band.FlushCache()

# Generates all the output png and tif images
def generate_SVI_images(files_list, svi, extent, geo_transform, projection):
    """
    Generates all the output png and tif images.

    :param files_list: list of files to be processed, containing a tuples of
                       the form (<str> file_path, <int> year, <str> day)
    :param svi: numpy array containing the svi
    :param extent:
    :param geo_transform: geo transform data to be used in saving the tif image
    :param projection: projection data to be used in saving the tif image
    :return:
    """
    day = files_list[0][2]
    years = [i[1] for i in files_list]

    # Calculate the standard deviation values that will be used to define the
    # color scheme
    std = np.nanstd(svi[:, :, 0])
    std15 = 1.5 * std
    std2 = 2 * std
    breaks = [-4, -std2, -std15, -std, std, std15, std2, 4]

    for i, year in enumerate(years, "Generating images"):
        file_name = "SVI_{}_{}_{}".format(study_area, day, year)
        array = svi[:, :, i]
        # Generating pngs for every time step
        # Use pre-defined function to write pngs
        generate_SVI_png(file_name, array, breaks, extent)

        # Generating geotiffs for every time step
        # Use pre-defined function to write geotiffs
        generate_geotiff(file_name, array, geo_transform, projection)
        gc.collect()  # Force garbage collection to save RAM

# Generates all the output png and tif images
def generate_VCI_images(files_list, vci, extent, geo_transform, projection):
    """
    Generates all the output png and tif images.

    :param files_list: list of files to be processed, containing a tuples of
                       the form (<str> file_path, <int> year, <str> day)
    :param vci: numpy array containing the vci
    :param extent:
    :param geo_transform: geo transform data to be used in saving the tif image
    :param projection: projection data to be used in saving the tif image
    :return:
    """
    day = files_list[0][2]
    years = [i[1] for i in files_list]
    for i, year in enumerate(years, "Generating images"):
        file_name = "VCI_{}_{}_{}".format(study_area, day, year)
        array = vci[:, :, i]
        # Generating pngs for every time step
        # Use pre-defined function to write pngs
        generate_VCI_png(file_name, array, extent)

        # Generating geotiffs for every time step
        # Use pre-defined function to write geotiffs
        generate_geotiff(file_name, array, geo_transform, projection)
        gc.collect()  # Force garbage collection to save RAM
   
# Checks folder structure
def filechooserCallback(chooser):
    
    global study_area
    global input_path
    global path_evi
    global path_pr
    global path_qual
    global output_path
    global path_png
    global path_tif
    
    # Specify the name of your study area, this will be used for naming the output maps
    study_area = chooser.selected_path.split('/')[-1]

    # check if input and output folders exist, if note create folders
    # input folders
    input_path = os.path.join(chooser.selected_path, 'input')
    if not os.path.isdir(input_path):
        os.mkdir(input_path)
    # folder for EVI geotiffs
    path_evi = os.path.join(input_path, 'evi_data')
    if not os.path.isdir(path_evi):
        os.mkdir(path_evi)
    # folder for pixel reliability geotiffs
    path_pr = os.path.join(input_path, 'pixel_reliability')
    if not os.path.isdir(path_pr):
        os.mkdir(path_pr)
    # folder for quality
    path_qual = os.path.join(input_path, 'quality')
    if not os.path.isdir(path_qual):
        os.mkdir(path_qual)

    # output folders
    output_path = os.path.join(chooser.selected_path, 'output')
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    # folder for png files
    path_png = os.path.join(output_path, 'SVI_png')
    if not os.path.isdir(path_png):
        os.mkdir(path_png)
    # folder for tif files
    path_tif = os.path.join(output_path, 'SVI_tif')
    if not os.path.isdir(path_tif):
        os.mkdir(path_tif) 


        
        
####################################################
####################### CODE #######################
####################################################  

fc = FileChooser(os.getcwd())
fc.title = '<b>Please select path to folder:</b>'
fc.show_only_dirs = True
fc.use_dir_icons = True
fc.register_callback(filechooserCallback)
display(fc)

download = ipywidgets.Dropdown(description = 'Data Download',
                               options     = ['Yes', 'No'],
                               layout      = ipywidgets.Layout(width='40%'),
                               style       = {'description_width': '15ex'})
display(download)

index = ipywidgets.Dropdown(description = 'Index',
                            options     = ['SVI', 'VCI'],
                            layout      = ipywidgets.Layout(width='40%'),
                            style       = {'description_width': '15ex'})
display(index)

## Download Data

<img src="https://github.com/vhertel/drought-monitoring/blob/main/resources/charts/chart2.png?raw=1" width="1000"/>

This section allows interactive data access and download from [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/). After entering the respective login details and the start date of the analysis, the desired country can be selected on the map. By clicking the *'Request'* button, the corresponding MODIS data will be queried and collected through [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/). Depending on the selected time period and country size, this process may take some minutes to hours. The status can be retrieved by clicking the *'Check Status'* button. The name of the folder selected above will be used as the title of the request. Once the request has been processed, the *'See Products'* button can be used to view a list of all queries on [AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/) and to download the data to the folder selected above.

In [ ]:
# Click to run

####################################################
############### FUNCTION DEFINITIONS ###############
####################################################

# Shows selected country/area
def click_handler(feature, **kwargs):
    html.value = '<h5>Selected: <b>{}</b></h5>'.format(feature['properties']['name'])

# Date conversion
def datestdtojd(stddate):
    fmt = '%d-%m-%Y'
    sdtdate = datetime.datetime.strptime(stddate, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday
    return(jdate)

# Requests task at AppEEARS
def on_requestButton_clicked(b, user, password):
    
    # Status update
    print('Product was requested. Please check status for data query progress.')
    
    # Insert API URL, call login service, provide credentials & return json
    token_response = r.post('{}login'.format(api), auth=(user.value, password.value)).json()
    # Remove user and password information        
    del user, password
    
    # Start a list for products to be requested, beginning with MOD13Q1.006
    prods = ['MOD13Q1.006']     
    # Request layers for the first product
    lst_response = r.get('{}product/{}'.format(api, prods[0])).json()  
    # Create tupled list linking desired product with desired layers
    layers = [(prods[0],'_250m_16_days_EVI')]  
    # Append to tupled list linking desired product with desired layers
    layers.append((prods[0],'_250m_16_days_pixel_reliability')) 
    prodLayer = []
    for l in layers:
        prodLayer.append({
                "layer": l[1],
                "product": l[0]
              })
        
    # Save login token to a variable    
    token = token_response['token']   
    # Create a header to store token information, needed to submit a request
    global head
    head = {'Authorization': 'Bearer {}'.format(token)}  
    
    # Checks whether country/area was selected
    country = re.search('<h5>Selected: <b>(.*)</b></h5>', html.value)
    if country is None:
        raise Exception('Please select country on the map.')
    # Extract area of interest and set to variable
    selected_country = countries.query('name == "%s"' % country.group(1)).to_json()      
    aoi_gc = json.loads(selected_country)

    # Call to spatial API, return projs as json
    projections = r.get('{}spatial/proj'.format(api)).json()  
    # Create an empty dictionary
    projs = {}                      
    # Fill dictionary with `Name` as keys
    for p in projections: 
        projs[p['Name']] = p                    
    
    # Create relevant dates required below
    today = date.today()
    # Convert to required format
    d1 = today.strftime("%m-%d-%Y")   
    #d1_day = datestdtojd(d1)
    #d1_year = today.strftime("%Y")
    #d1_day_jcd = d1_year + str(d1_day)
    
    # Create list with existing png files
    file_list_folder = [f for f in os.listdir(path_png) if fnmatch.fnmatch(f, '*.png')] 
    # If files already exist, extract date of late file 
    if len(file_list_folder) != 0:            
        # Sort files in file list according to year and doy.
        s = sorted(file_list_folder, key = lambda x: (x.split('_')[-1],x.split('.')[0])) 
        # Extract date of last png in the folder.
        last_jcd = s[-1].split('.')[0].split('_')[-1]+s[-1].split('_')[-2] 
        # Extract day of last png in folder
        last_day = s[-1].split('_')[-2]         
        # Extract day of last png in folder
        last_year = s[-1].split('.')[0].split('_')[-1]     
        # Convert date to normal datetime
        d = datetime.datetime.strptime(last_jcd, '%Y%j').date()       
        # Add 16 days as next image will be available for that date.
        nd = d + datetime.timedelta(days=16)                               
        
        # Format to meet api date format.
        last_image = d.strftime("%m-%d-%Y")                                
        next_image = nd.strftime("%m-%d-%Y")

        # Status update
        print(f'The last image in the folder is from {last_image}. As a result, the start date for the request will be {next_image}.')

    else:
        # If list does not exist, start from first date for which data is available.
        last_jcd = '0'                        
        next_image = str(datetime.datetime.strptime(str(start_date_picker.value), '%Y-%m-%d').strftime('%m-%d-%Y'))
 
    # Type of task, area or point
    task_type = ['point','area']   
    # Set output projection 
    proj = projs['geographic']['Name']  
    # Set output file format type
    outFormat = ['geotiff', 'netcdf4']  
    # Start of the date range for which to extract data: MM-DD-YYYY
    startDate = next_image              
    # End of the date range for which to extract data: MM-DD-YYYY. Set as d1 to use date of last image available in folder
    endDate = d1                 
    # Specify True for a recurring date range
    recurring = False                   
    # if recurring = True, set yearRange, change start/end date to MM-DD
    #yearRange = [2000,2016]            

    task = {
        'task_type': task_type[1],
        'task_name': task_name,
        'params': {
             'dates': [
             {
                 'startDate': startDate,
                 'endDate': endDate
             }],
             'layers': prodLayer,
             'output': {
                     'format': {
                             'type': outFormat[0]}, 
                             'projection': proj},
             'geo': aoi_gc,
        }
    }
    
    # Post json to the API task service, return response as json
    global task_response
    task_response = r.post('{}task'.format(api), json=task, headers=head).json()  

    # enable button to check status
    checkStatusButton.disabled = False    

# Checks status on submitted task
def on_checkStatusButton_clicked(b):
    task_id = task_response['task_id']
    print('Status: %s' % r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

# Displays all available data queries
def on_productsButton_clicked(b, user, password):

    # Insert API URL, call login service, provide credentials & return json
    token_response = r.post('{}login'.format(api), auth=(user.value, password.value)).json() 
    # Remove user and password information        
    del user, password         
    # Save login token to a variable
    token = token_response['token']    
    # Create a header to store token information, needed to submit a request
    head = {'Authorization': 'Bearer {}'.format(token)}  

    # Query task service, setting params and header 
    tasks_response = r.get('{}task'.format(api), headers=head).json() 
    # print table with download buttons
    grid = ipywidgets.GridspecLayout(len(tasks_response)+1, 4)
    grid[0,0] = ipywidgets.HTML('<h4>Name</h4>')
    grid[0,1] = ipywidgets.HTML('<h4>Requested on</h4>')
    grid[0,2] = ipywidgets.HTML('<h4>Status</h4>')
    for i in range(len(tasks_response)):
        grid[i+1,0] = ipywidgets.Label(tasks_response[i]['task_name'])
        grid[i+1,1] = ipywidgets.Label(tasks_response[i]['created'])
        grid[i+1,2] = ipywidgets.Label(tasks_response[i]['status'])
        grid[i+1,3] = ipywidgets.Button(description = 'Download')
        grid[i+1,3].on_click(functools.partial(on_downloadButton_clicked, tasks_response=tasks_response[i]))
    display(grid)
    
# Downloads data
def on_downloadButton_clicked(b, tasks_response):
        
    # Checks status of requested dataset
    if tasks_response['status'] == 'done':    

        task_id = tasks_response['task_id']
        # Call API and return bundle contents for the task_id as json
        bundle = r.get('{}bundle/{}'.format(api, task_id)).json()  
        # Create empty dictionary
        files = {}            
        # Fill dictionary with file_id as keys and file_name as values
        for f in bundle['files']: 
            files[f['file_id']] = f['file_name']                        
        # Instantiates progress bar
        progressBar = ipywidgets.IntProgress(description = 'Downloading %d images:' % len(files),
                                             min         = 0,
                                             max         = len(files),
                                             layout      = ipywidgets.Layout(width='60%'),
                                             style       = {'description_width': '25ex'})    
        # display the bar
        display(progressBar)                                           
            
        for f in files:
            # Get a stream to the bundle file
            dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), stream=True)    
            # Parse the name from Content-Disposition header 
            filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  
            if fnmatch.fnmatch(filename, '*EVI*'):
                filepath = os.path.join(path_evi, filename)
            elif fnmatch.fnmatch(filename,'*pixel_reliability*'):
                filepath = os.path.join(path_pr, filename)
            elif fnmatch.fnmatch(filename,'*Quality*'):
                filepath = os.path.join(path_qual, filename)
            else:
                filepath = os.path.join(input_path, filename)
            # Write file to dest dir
            with open(filepath, 'wb') as f:                                                                  
                for data in dl.iter_content(chunk_size=8192): 
                    f.write(data) 
            progressBar.value += 1
        print('Downloaded files can be found at: {}'.format(input_path))

    else:
        print('Data query not complete. Please check status.')



####################################################
####################### CODE #######################
####################################################
        
# Checks whether data shall be downloaded
if download.value == 'Yes':
    # Creates interactive map for country/area selection
    m = ipyleaflet.Map(zoom = 2, basemap = ipyleaflet.basemaps.CartoDB.Positron)
    countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    geo_data = ipyleaflet.GeoData(geo_dataframe = countries,
                                  style={'opacity' : 0, 'fillOpacity' : 0},
                                  hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                                  name = 'Countries')
    m.add_layer(geo_data)
    tile_ID = geo_data.on_click(click_handler)
    html = ipywidgets.HTML('')
    control = ipyleaflet.WidgetControl(widget=html, position='topright')
    m.add_control(control)
    display(m)

    # Give the project a name. This will be used throughout the script.
    # User-defined name of the task
    task_name = study_area 

    # Change to working directory
    os.chdir(input_path)                     
    # Set the AρρEEARS API to a variable
    api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'  
    # Input NASA Earthdata Login Username
    user = ipywidgets.Text(description = 'Username')      
    # Input NASA Earthdata Login Password
    password = ipywidgets.Password(description = 'Password')  
    start_date_picker = ipywidgets.DatePicker(description='Start Date:', value=datetime.datetime(2000,1,1).date())
    requestButton = ipywidgets.Button(description = 'Request')
    requestButton.on_click(functools.partial(on_requestButton_clicked, user=user, password=password))
    productsButton = ipywidgets.Button(description = 'See Products')
    productsButton.on_click(functools.partial(on_productsButton_clicked, user=user, password=password))
    checkStatusButton = ipywidgets.Button(description = 'Check Status', disabled=True)
    checkStatusButton.on_click(on_checkStatusButton_clicked)
    display(ipywidgets.HBox([user, password, start_date_picker]))    
    display(ipywidgets.HBox([requestButton, checkStatusButton, productsButton]))

## Processing

<img src="https://github.com/vhertel/drought-monitoring/blob/main/resources/charts/chart2.png?raw=1" width="1000"/>

### Standardized Vegetation Index (SVI)

Originally building on the NDVI anomaly concept, the Standardized Vegetation Index (SVI) developed by Peters et al. (2002) describes the probability of variation from the normal NDVI over multiple years of data (e.g. 12 years) on a weekly time step. The SVI is a z-score deviation from the mean in units of the standard deviation, calculated from the NDVI or EVI values for each pixel location of a composite period for each year during a given reference period. The equation below shows the general calculation of the SVI

$$z_{ijk} = \frac{\text{VI}_{ijk} - \mu_{ij}}{\sigma_{ij}}$$

where $z_{ijk}$ is the z-value for the pixel $i$ during week $j$ for year $k$, $\text{VI}_{ij}$ is the weekly $\text{VI}$ value for pixel $i$ during week $j$ for year $k$ whereby both the NDVI or EVI can be utilized as $\text{VI}$, $\mu_{ij}$ is the mean for pixel $i$ during week $j$ over $n$ years, and $\sigma_{ij}$ is the standard deviation of pixel $i$ during week $j$ over $n$ years. This recommended practice calculates the SVI based on the Enhanced Vegetation Index (EVI) which has some advantages compared to the NDVI such as an improved sensitivity over dense vegetation conditions and is less affected by aerosol influences. More detailed information on the Standardized Vegetation Index can be found [here](https://www.un-spider.org/advisory-support/recommended-practices/recommended-practice-agricultural-drought-monitoring-svi/in-detail/standardized-vegetation-index).

### Vegetation Condition Index (VCI)

Kogan proposed a Vegetation Condition Index (VCI) based on the relative Normalized Difference Vegetation Index (NDVI) change with respect to minimum historical NDVI value. The VCI therefore compares the current Vegetation Index (VI) such as NDVI or Enhanced Vegetation Index (EVI) to the values observed in the same period in previous years within a specific pixel. The VCI is calculated as shown below,

$$ \text{VCI}_{ijk} = \frac{\text{VI}_{ijk} - \text{VI}_{i,min}}{\text{VI}_{i,max} - \text{VI}_{i,min}} \cdot 100$$

where $\text{VCI}_{ijk}$ is the $\text{VCI}$ value for the pixel $i$ during week/month/DOY $j$ for year $k$, $\text{VI}_{ijk}$ is the weekly/monthly/DOYs $\text{VI}$ value for pixel $i$ in week/month/DOY $j$ for year $k$ whereby both the NDVI or EVI can utilized as $\text{VI}$, $\text{VI}_{i,min}$ and $\text{VI}_{i,max}$ are the multi-year minimum and maximum $\text{VI}$, respectively, for pixel $i$.

The resulting percentage of the observed value is situated between the extreme values (minimum and maximum) in the previous years. Lower and higher values, therefore, indicate bad and good vegetation state conditions, respectively.

In [ ]:
# Click to run

####################################################
####################### CODE #######################
####################################################

# The following regular expression will match 'doy' followed by 7 digits,
# followed by 0 or more characters and ending in '.tif'
# It is used for building the initial file list of tif images that have a valid
# DOY structure
re_doy = re.compile(r".*doy(\d{7}).*\.tif$")

# Build initial file list of tuples containing the filename, year and day for
# each file
# e.g. [(<str> '/path/to/my_evi_doy2000193.tif', <int> 2000, <str> '193')]
# Note: the data types are stated in '<>'
evi_files = []

# Create a list of files, which include the defined DOY in their filename
# for the EVI data
for _, _, files in os.walk(path_evi):
    for file in files:
        doy = get_doy(re_doy, file)
        if doy is None:
            continue
        else:
            evi_files.append((os.path.join(path_evi, file), doy[0], doy[1]))

# Create a list of files, which include the defined DOY in their filename
# for the pixel reliability data
pr_files = []

for _, _, files in os.walk(path_pr):
    for file in files:
        doy = get_doy(re_doy, file)
        if doy is None:
            continue
        else:
            pr_files.append((os.path.join(path_pr, file), doy[0], doy[1]))

# Read an example file and define the shape of the data arrays
# Get the first file of the file list as example file
example_file = gdal.Open(evi_files[0][0])

# Store necessary data from the example file
geo_transform = example_file.GetGeoTransform()
projection = example_file.GetProjection()
x_size = example_file.RasterXSize
y_size = example_file.RasterYSize

del example_file  # Save RAM

# Preparing map reshaping
lon_start = geo_transform[0]
lon_stop = (x_size * geo_transform[1]) + geo_transform[0]
lon_step = geo_transform[1]
lat_start = geo_transform[3]
lat_stop = (y_size * geo_transform[5]) + geo_transform[3]
lat_step = geo_transform[5]

extent = (lon_start, lon_stop, lat_stop, lat_start)

# Build a dictionary where keys are the available DOYs and values are sorted
# dictionaries for evi files and pr files
doy_dict = check_prepare_files(evi_files, pr_files)

# The script creates some warnings due to the NA in the data.
# They are ignored by executing this cell.
warnings.filterwarnings('ignore')

# If tqdm is available, use it.
# Note: both update_iter_desc and days_iterator are set in either case since it
# allows to avoid having many if clauses throughout the program and reduces
# code duplication.
if tqdm is not None:
    def update_iter_desc(days_iterator, desc):
        days_iterator.set_description(desc)
    days_iterator = tqdm(sorted(doy_dict), desc="Processing",
                         file=sys.stdout)
else:
    def update_iter_desc(days_iterator, desc):
            print(desc)
    days_iterator = sorted(doy_dict)

# Begin iterating through days of the year for which we have data.
for day in days_iterator:
    # Provide a progress update
    update_iter_desc(days_iterator, "Processing DOY {}".format(day))

    # Get the necessary files lists from doy_dict
    evi_files_day = doy_dict[day]['evi']
    pr_files_day = doy_dict[day]['pr']

    # Number of years for which we have data
    num_years = len(evi_files_day)

    # Create a zero-filled 3D numpy array based on the example file
    # dimensions
    array_size = (y_size, x_size, num_years)
    try:
        # Adjust the size of the cloud mask array in place in order to save
        # RAM and speed up processing
        cloud_mask.resize(array_size)
    except NameError:
        # If this is the first iteration, cloud_mask will be undefined and
        # trying to resize it will throw a NameError. This error is caught
        # and cloud_mask is instantiated
        cloud_mask = np.zeros(array_size, dtype=np.float64)

    # Reading the reliability data
    cloud_mask = load_cloud_mask(pr_files_day, cloud_mask)

    # Preparing the reliability data
    cloud_mask = prepare_cloud_mask(cloud_mask)

    # Reading the EVI  data
    evi = load_evi(evi_files_day, cloud_mask)

    # Preparing the EVI  data
    evi = prepare_evi(evi)
    
    if index.value == 'SVI':
        update_iter_desc(days_iterator, "Calculating SVI DOY {}".format(day))
        # Calculate SVI
        svi = calculate_svi(evi)

        update_iter_desc(days_iterator, "Generating images DOY {}".format(day))
        # Generating png images and geotiffs
        generate_SVI_images(evi_files_day, svi, extent, geo_transform, projection)

        # Remove references to avoid array resize errors in future loops
        del(evi, svi)
    elif index.value == 'VCI':
        update_iter_desc(days_iterator, "Calculating VCI DOY {}".format(day))
        # Calculate VCI
        vci = calculate_vci(evi)

        update_iter_desc(days_iterator, "Generating images DOY {}".format(day))
        # Generating png images and geotiffs
        generate_VCI_images(evi_files_day, vci, extent, geo_transform, projection)

        # Remove references to avoid array resize errors in future loops
        del(evi, vci)
    
# create gif
file_list = os.listdir(path_png)
s = sorted(file_list, key = lambda x: (x.split('_')[-1],x.split('.')[0]))
images = []
for file_name in s:
    if file_name.endswith('.png'):
        file_path = os.path.join(path_png, file_name)
        images.append(imageio.imread(file_path))

gif_path = os.path.join(output_path, study_area+'_svi_animation.gif')
imageio.mimsave(gif_path, images)

# create grid image
first_year = int(s[0].split('.')[0].split('_')[-1])
last_year = int(s[-1].split('.')[0].split('_')[-1])
# correlation between DOY and grid row
row_dict = {
                '001': 0,
                '017': 1,
                '033': 2,
                '049': 3,
                '065': 4,
                '081': 5,
                '097': 6,
                '113': 7,
                '129': 8,
                '145': 9,
                '161': 10,
                '177': 11,
                '193': 12,
                '209': 13,
                '225': 14,
                '241': 15,
                '257': 16,
                '273': 17,
                '289': 18,
                '305': 19,
                '321': 20,
                '337': 21,
                '353': 22,
}
# correlation between year and grid column
col_dict = {}
for year in range(first_year, last_year+1):
    col_dict[str(year)] = year - first_year

# number of rows and columns
nrows = len(row_dict)
ncols = last_year-first_year+1

# figure
fig = plt.figure(figsize=(ncols, nrows))
spec = gridspec.GridSpec(nrows=nrows, ncols=ncols, figure=fig, wspace=0.05, hspace=0.05)
# inserts png images
i=0
for file_name in s:
    row = row_dict[file_name.split('_')[2]]
    col = col_dict[file_name.split('.')[0].split('_')[-1]]
    ax1 = fig.add_subplot(spec[row, col], frameon=False)
    ax1.set(aspect = 'equal')
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.imshow(images[i])
    i = i + 1
# inserts DOYs for first column
j = 0
for key, value in row_dict.items():
    ax2 = fig.add_subplot(spec[j, 0], frameon=False)
    ax2.set_ylabel('DOY %s' % key, rotation=90)
    ax2.set_xticks([])
    ax2.set_yticks([])
    j = j + 1
j = 0
# inserts years for first row
for key, value in col_dict.items():
    ax3 = fig.add_subplot(spec[0, j], frameon=False)
    ax3.set_title(key)
    ax3.set_xticks([])
    ax3.set_yticks([])
    j = j + 1
# saves figure
grid_path = os.path.join(output_path, study_area+'_grid_figure.png')
plt.savefig(grid_path, dpi=400, bbox_inches='tight', edgecolor='none', facecolor='w')